In [109]:
from pathlib import Path
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib
import torch

from zunis.utils.config.loaders import get_default_integrator_config
from zunis.utils.config.loaders import create_integrator_args
from utils.integrands.camel import KnownSymmetricCamelIntegrand

In [2]:
here = Path('.').resolve()
benchmarks = (here / '..').resolve()
print(benchmarks)
sys.path.append(str(benchmarks))

plt.rcParams.update({
    'font.size': 8,
    'text.usetex': True,
    'text.latex.preamble': r'\usepackage{amsfonts}',
    "figure.figsize": (6,4),
    'figure.dpi':150
})

sns.set_theme()
sns.set(font_scale=1.3)
sns.set_style("whitegrid")

/scratch/ndeutsch/zunis/zunis/experiments/benchmarks


In [3]:
from utils.data_storage.dataframe2sql import read_pkl_sql
from utils.config.loaders import get_sql_types
import sqlite3 as sql
from sqlalchemy import PickleType


In [4]:
con = sql.connect("benchmarks.db")
con.cursor().execute("SELECT name FROM sqlite_master where type = 'table'").fetchall()

[('camel_defaults',)]

In [5]:
dtypes = get_sql_types()
dtypes["value_history"] = PickleType

df = read_pkl_sql("benchmarks.db", "camel_defaults", dtypes=dtypes)
df.columns = df.columns.astype(str)
df = df.loc[df.lr == 1.e-3]
d_sigmas = pd.read_csv('../../exploration/gaussian_camel_integrands.csv')
d_sigma_camel = d_sigmas[['d','sigma_camel','sigma_1d','relative_std_camel']].rename(columns=
                                                                     {
                                                                         "sigma_camel":'s',
                                                                         'relative_std_camel':'relative_std'
                                                                     })

df = df.merge(d_sigma_camel,on=["d","s"],how="left")

In [81]:
cpts = df.groupby('d')['checkpoint_path'].max()
cpts = cpts.apply(lambda x : ('..' / Path(x)).resolve())

In [88]:
cpts[8]

PosixPath('/scratch/ndeutsch/zunis/zunis/experiments/benchmarks/benchmarks_04/camel/camel8d.ckpt')

In [22]:
from zunis.integration import Integrator

In [63]:
config = create_integrator_args(get_default_integrator_config())

In [68]:
config

{'flow': 'pwquad',
 'loss': 'variance',
 'n_points_survey': 10000,
 'survey_strategy': 'adaptive_dkl',
 'n_iter': 10,
 'flow_options': {'masking': 'iflow',
  'cell_params': {'d_hidden': 256, 'n_bins': 10, 'n_hidden': 8},
  'masking_options': {'repetitions': 2}},
 'trainer_options': {'minibatch_size': 1.0,
  'max_reloads': 0,
  'n_epochs': 50,
  'checkpoint': True,
  'checkpoint_on_cuda': True,
  'checkpoint_path': None,
  'optim': functools.partial(<class 'torch.optim.adam.Adam'>, betas=(0.9, 0.999), eps=1e-08, lr=0.001)}}

In [111]:
df.loc[df.d == 8].iloc[0]['s']

0.5225006639951927

In [113]:
f = KnownSymmetricCamelIntegrand(d=8, s=df.loc[df.d == 8].iloc[0]['s'])
integ = Integrator(d=8,f=f, **config)

In [114]:
integ.model_trainer.flow.load_state_dict(dd)

<All keys matched successfully>

In [117]:
integ.integrate(n_survey_steps=0, n_refine_steps=2)

(0.0868079077738982,
 1.0882541240134783e-05,
     integral     error  n_points   phase
 0   0.086820  0.000039    100000  refine
 1   0.086762  0.000039    100000  refine
 2   0.086855  0.000039    100000  refine
 3   0.086873  0.000039    100000  refine
 4   0.086819  0.000039    100000  refine
 5   0.086735  0.000039    100000  refine
 6   0.086824  0.000039    100000  refine
 7   0.086753  0.000039    100000  refine
 8   0.086814  0.000039    100000  refine
 9   0.086836  0.000039    100000  refine
 10  0.086825  0.000039    100000  refine
 11  0.086761  0.000039    100000  refine
 12  0.086826  0.000039    100000  refine)

In [104]:
p=integ.model_trainer.flow.parameters()

In [108]:
next(p).device

device(type='cpu')